In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import f_regression
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns

from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

data_df = None
if os.path.isfile("data/data.pkl"):
    data_df = pd.read_pickle("data/data.pkl")
else:
    print("Load Data")
data_df.head()

RANDOM_STATE = 42

In [ ]:
# Define X and Y
data_df["Sex_binary"] = data_df["Sex"].map({"M": 0, "m": 0, "F": 1, "f": 1})
ad_hc_df = data_df[
    data_df["Diagnosis"].isin(["AD", "HC"]) & (data_df["Harmonized"].notna())
].copy()
tbi_df = data_df[
    data_df["Diagnosis"].isin(["NEG", "POS"]) & (data_df["Harmonized"].notna())
].copy()

X_ad_hc_evc = StandardScaler().fit_transform(np.vstack(ad_hc_df["EVC"].values))
age_ad_hc = ad_hc_df["Age"].values.reshape(-1, 1)
sex_ad_hc = ad_hc_df["Sex_binary"].values.reshape(-1, 1)

X_ad_hc = np.hstack([X_ad_hc_evc, age_ad_hc, sex_ad_hc])
y_ad_hc = ad_hc_df["Diagnosis"].map({"AD": 1, "HC": 0}).values

X_tbi_evc = StandardScaler().fit_transform(np.vstack(tbi_df["EVC"].values))
age_tbi = tbi_df["Age"].values.reshape(-1, 1)
sex_tbi = tbi_df["Sex_binary"].values.reshape(-1, 1)

X_tbi = np.hstack([X_tbi_evc, age_tbi, sex_tbi])
y_tbi = tbi_df["Diagnosis"].map({"POS": 1, "NEG": 0}).values

In [ ]:
def run_ridge_cv(X, y):
    model = RidgeCV(alphas=[0.1, 1.0, 10.0])
    model.fit(X, y)
    return model.coef_


coef_ad_hc = run_ridge_cv(X_ad_hc, y_ad_hc)
coef_tbi = run_ridge_cv(X_tbi, y_tbi)
# coef_mci = run_ridge_cv(X_mci, y_mci)

coef_df = pd.DataFrame(
    {
        "Feature": [f"Feature_{i}" for i in range(X_ad_hc.shape[1])],
        "AD_HC": coef_ad_hc,
        "TBI": coef_tbi,
        # "MCI": coef_mci
    }
)

coef_df_melted = coef_df.melt(
    id_vars=["Feature"], var_name="Diagnosis", value_name="Coefficient"
)

fig, axes = plt.subplots(3, 1, figsize=(12, 18))


def set_xticks(ax, num_ticks, total_ticks):
    step_size = total_ticks // num_ticks
    ticks_to_show = list(range(0, total_ticks, step_size))
    ax.set_xticks(ticks_to_show)
    ax.set_xticklabels([f"Feature_{i}" for i in ticks_to_show], rotation=45, ha="right")


sns.boxplot(
    x="Feature",
    y="Coefficient",
    hue="Diagnosis",
    data=coef_df_melted[coef_df_melted["Diagnosis"] == "AD_HC"],
    ax=axes[0],
    showfliers=False,
)
axes[0].set_title("Coefficient Values for AD_HC")
axes[0].set_xlabel("Feature")
axes[0].set_ylabel("Coefficient Value")
set_xticks(axes[0], 10, len(coef_df))
axes[0].grid(True, which="major", axis="both", linestyle="--", linewidth=0.2)

sns.boxplot(
    x="Feature",
    y="Coefficient",
    hue="Diagnosis",
    data=coef_df_melted[coef_df_melted["Diagnosis"] == "TBI"],
    ax=axes[1],
    showfliers=False,
)
axes[1].set_title("Coefficient Values for TBI")
axes[1].set_xlabel("Feature")
axes[1].set_ylabel("Coefficient Value")
set_xticks(axes[1], 10, len(coef_df))
axes[1].grid(True, which="major", axis="both", linestyle="--", linewidth=0.2)

sns.boxplot(
    x="Feature",
    y="Coefficient",
    hue="Diagnosis",
    data=coef_df_melted[coef_df_melted["Diagnosis"] == "MCI"],
    ax=axes[2],
    showfliers=False,
)
axes[2].set_title("Coefficient Values for MCI")
axes[2].set_xlabel("Feature")
axes[2].set_ylabel("Coefficient Value")
set_xticks(axes[2], 10, len(coef_df))
axes[2].grid(True, which="major", axis="both", linestyle="--", linewidth=0.2)

plt.tight_layout()
plt.show()

In [ ]:
def highlightInformative(x, y, axis):
    residuals = np.abs(y - x)
    magnitudes = np.abs(x) + np.abs(y)
    weights = -1 * residuals + magnitudes
    top_10_indices = np.argsort(weights)[-10:][::-1]
    axis.scatter(x[top_10_indices], y[top_10_indices], alpha=0.9, c="red", marker="2")
    for idx in top_10_indices:
        print(f"{idx}\t{weights[idx]}")
    print("==========\n")


def calculate_r2(x, y):
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    y_pred = model.predict(x.reshape(-1, 1))
    return r2_score(y, y_pred)


fig, axes = plt.subplots(1, 3, figsize=(10, 12))

# r2_tbi_mci = calculate_r2(coef_mci, coef_tbi)
# axes[0].scatter(coef_mci, coef_tbi, alpha=0.7, c="black", marker="2")
# axes[0].plot([-1, 1], [-1, 1], linestyle="--", color="black", alpha=0.5)
# axes[0].set_xlabel("Regression Coefficient")
# axes[0].set_ylabel("Regression Coefficient")
# axes[0].set_title(f"TBI vs MCI (R^2 = {r2_tbi_mci:.2e})")
# axes[0].grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
# axes[0].set_xlim([-0.15, 0.15])
# axes[0].set_ylim([-0.15, 0.15])
# axes[0].set_aspect("equal", adjustable="box")
# highlightInformative(coef_mci, coef_tbi, axes[0])

r2_ad_hc_tbi = calculate_r2(coef_ad_hc, coef_tbi)
axes[1].scatter(coef_ad_hc, coef_tbi, alpha=0.7, c="black", marker="2")
axes[1].plot([-1, 1], [-1, 1], linestyle="--", color="black", alpha=0.5)
axes[1].set_xlabel("Regression Coefficient")
axes[1].set_ylabel("Regression Coefficient")
axes[1].set_title(f"TBI vs AD (R^2 = {r2_ad_hc_tbi:.2e})")
axes[1].grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
axes[1].set_xlim([-0.15, 0.15])
axes[1].set_ylim([-0.15, 0.15])
axes[1].set_aspect("equal", adjustable="box")
highlightInformative(coef_ad_hc, coef_tbi, axes[1])

# r2_ad_hc_mci = calculate_r2(coef_ad_hc, coef_mci)
# axes[2].scatter(coef_ad_hc, coef_mci, alpha=0.7, c="black", marker="2")
# axes[2].plot([-1, 1], [-1, 1], linestyle="--", color="black", alpha=0.5)
# axes[2].set_xlabel("Regression Coefficient")
# axes[2].set_ylabel("Regression Coefficient")
# axes[2].set_title(f"MCI vs AD (R^2 = {r2_ad_hc_mci:.2e})")
# axes[2].grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
# axes[2].set_xlim([-0.15, 0.15])
# axes[2].set_ylim([-0.15, 0.15])
# axes[2].set_aspect("equal", adjustable="box")
# highlightInformative(coef_ad_hc, coef_mci, axes[2])

plt.tight_layout()
plt.show()

In [ ]:
df = pd.DataFrame(data_df)
df = df.drop(columns=["Data", "Connectome", "Harmonized", "CLU", "DIV"])
expanded_columns = []

for idx, row in df.iterrows():
    evc_matrix = row["EVC"]

    if evc_matrix is not None:
        expanded_columns.append(evc_matrix.flatten())
    else:
        expanded_columns.append([np.nan] * 4)

evc_expanded = pd.DataFrame(expanded_columns, index=df.index)

evc_expanded.columns = [f"EVC_{i}" for i in range(evc_expanded.shape[1])]

df = pd.concat([df.drop(columns=["EVC"]), evc_expanded], axis=1)

df.to_csv("expanded_dataframe.csv", index=False)

In [ ]:
f_stat_ad, p_values_ad = f_regression(X_ad_hc, y_ad_hc)
f_stat_tbi, p_values_tbi = f_regression(X_tbi, y_tbi)

_, p_values_ad_hc_corrected, _, _ = multipletests(p_values_ad, method="bonferroni")
_, p_values_tbi_corrected, _, _ = multipletests(p_values_tbi, method="bonferroni")

significant_ad_hc = np.where(p_values_ad_hc_corrected < 0.05)[0]
significant_tbi = np.where(p_values_tbi_corrected < 0.05)[0]

print(f"P Values AD/HC: {p_values_ad}", end="\n========\n\n")
print(f"P Values TBI+/TBI-: {p_values_tbi}", end="\n========\n\n")

In [ ]:
significant_ad_hc